# Generative Agents Notebook

This is a notebook demonstrating most of the concepts in the paper Generative Agents: Interactive Simulacra of Human Behavior (https://arxiv.org/abs/2304.03442)
Builds on https://github.com/mkturkcan/generative-agents

## Notes

This notebook relies on an OpenAI API compatible LM server. To use a model directly, e.g. from Huggingface transformers, redefine the generate() function.

It is possible to impersonate an agent by redirecting LM calls to the user. See the generate() function.

## The Simulation

Simulates a fictional university environment, with locations defined as a NetworkX graph. Basic time unit is an hour, and agents observe actions of other agents in the same location. Each hour, each agent:
* Determines what to do in the next hour
  * If a chat with another agent is implied, the model is asked for a transcript
* Collects, compresses, and ranks memories
* Decides whether to move to a new location

## Improvements

* Support more than two people in conversations
* Add recency to memories
* RAG for memories?
* Support different time scales and multiple days
* Add daily reflections

In [ ]:
import networkx as nx
import re
from openai import OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

In [ ]:
system_prompt = "Reply in character. "
prompt_format = '''### Instruction:
{}

### Response:
'''
prompt_format = '{}'   #empty if LM server takes care of it

def generate(prompt, name=None, user=None):
    """Call on model with input prompt.
    To impersonate an agent and provide interactive output, change a generate call so that name is the name of the current agent and user is the name of the agent you want to impersonate."""
    if name and user:
        if name==user:
            print(prompt)
            return input()
    completion = client.chat.completions.create(
      model="local-model", # this field is currently unused
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_format.format(prompt)}
      ],
      temperature=0.8,
    )
    
    return completion.choices[0].message.content

## World Model
The world is modelled as an undirected graph.

In [ ]:
world_graph = nx.Graph()

town_areas = {"The corridor": "A university corridor with several offices. On one end, the corridor leads to the staff room, and the other leads to a classroom.",
              "Daniel's office": "A sparsely decorated office where Daniel spends most of his time at work. Somehow it has more computers than desks.",
              "Alice's office": "Here Alice works surrounded by her books, from floor to ceiling, in several bookshelves.",
              "Bob's office": "Aside from the regular desk setup, Bob has a fish tank in his office.",
              "Ben's office": "Ben's office only has the bare necessities for university work.",
              "Eve's office": "Eve prefers standing and thus her desk is always raised. Her desk is also very cluttered, always.",
              "John's office": "This office where John sits is filled with all sorts of things, so much that you barely have room to visit.",
              "The staff room": "The staff room is where the staff go to relax and drink tea or coffee, which they usually do at 9 in the morning.",
              "Classroom": "This is a nice modern classroom supporting both on-campus and remote education thanks to the wonders of modern technology."
}
town_people = {"Daniel": "Daniel is a senior lecturer. He is trying his best to balance research and teaching. He is teaching a course on business intelligence, and he is trying to collaborate on AI research projects with government agencies.",
               "Alice": "Alice is a full professor, and on a paper grind. She is the one pulling in funding for the subject, and is deep in the strategic academic game. She also teaches data science courses.",
               "Bob": "Bob is a PhD student in his fourth year. His journey hasn't been easy, and he is frustrated at his latest project not giving him enough for his dissertation.",
               "Ben": "Ben is a first year PhD student barely up and running. He has trouble forming any strategic plan, as it feels like his project keeps shifting all the time.",
               "Eve": "Eve is a senior lecturer who focuses on teaching. She has done research before, but as of late it's the teaching that drives her.",
               "Lily": "Lily is a new employee. She has no office yet, and needs assistance getting up and running.",
               "John": "John is the head of division, supporting the running of the day-to-day business. He is currently talking with employees about their work environment in order to suggest improvements.",
               "Kevin": "Kevin is a cyber criminal who is now acting as journalist, inteding to interview people about their research. His goal is to gather as many research secrets as he can. He is desperate, and is willing to utilise unconventional means to accomplish his goal."
}

for town_area in town_areas.keys():
  world_graph.add_node(town_area)

for town_area in town_areas.keys():
  if town_area != "The corridor":  
    world_graph.add_edge(town_area, "The corridor")
locations = {}
for i in town_people.keys():
  locations[i] = "The corridor"

memories = {}
chats = {}
plans = {}
for i in town_people.keys():
  memories[i] = []
  chats[i] = []
  plans[i] = []

global_time = 8

nx.draw(world_graph, with_labels=True)

In [ ]:
compressed_memories_all = {}
chats_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []
  chats_all[name] = []

In [ ]:
for name in town_people.keys():
  prompt = f"You are acting as {name}. {town_people[name]} You just arrived for the working day in your corridor at the university. The following people are also here: {', '.join(list(town_people.keys()))}. Write what your goal for today is. Start with the word 'to', and use at most 15 words."
  #print(prompt)
  plans[name] = generate(prompt)
  print('['+name+']: ', plans[name])

In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. You are currently in {location} with the following description: {town_areas[location]}. It is currently {global_time}:00. The following people are in this area: {', '.join([p for p in people if p!=name])}. You can interact with them."
    people_description = []
    for i in people:
      if i != name:
        people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    prompt = prompt.replace('..', '.')
    #print(prompt+'\n')
    action_prompts[name] = prompt

In [ ]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(action_prompts[name])
  prompt = f"""
  Convert the following paragraph to first person past tense:
  "{action_results[name]}"
  """
  action_results[name] = generate(prompt).replace('"', '').replace("'", '')
  print('['+name+']: ', action_results[name])

In [ ]:
talked_to = []
for name in town_people.keys():
  if name in talked_to:
    continue
  a = action_results[name]
  #prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. The following is your recent action: {a}. Does this action imply talking with anyone? If so, choose from the following list the person you talked to, answer with name only, and if you talked to many, choose one of these: {', '.join([p for p in people if p!=name])}"
  prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. Your recent action is: {a}. Here is a list of people you could have talked to: {', '.join([p for p in people if p!=name])}. If you did not talk to anyone during the recent action, reply with 'None'. If you talked with anyone of these people during the recent action, reply with their name only. The reply must be a name from this given list."
  #print(prompt)
  x = generate(prompt)
  print(name+': '+x)
  names = [i for i in re.findall('|'.join(town_people.keys()), x)]
  #print(names)
  if names:
    prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. The following is your recent action: {a}. Since you talked with {names[0]}, write a transcript of the conversation. Keep it to 300 words."
    c = generate(prompt)
    print(f'[TRANSCRIPT OF {name} AND {names[0]}]')
    print(c+'\n')
    talked_to.append(names[0])
    chats[name].append((names[0], f'[Time: {global_time}. Chat between {name} and {names[0]}:\n{c}]\n'))
    chats[names[0]].append((name, f'[Time: {global_time}. Chat between {names[0]} and {name}:\n{c}]\n'))

## Compress chats into personal memories and store them
Currently supports only one chat per person and hour!!!

In [ ]:
for name in chats.keys():
    if not chats[name]:
        continue
    prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. You just had the conversation below with {chats[name][-1][0]}. Summarise what you personally got out of it, in one sentence. {chats[name][-1][1]}"
    s = generate(prompt)
    print(f'Memory of {name} chatting with {chats[name][-1][0]}\n{s}')
    memories[name].append(f'[Time: {global_time}. Person: {chats[name][-1][0]}. Memory: {s}]\n')

## Collect the memories people observe

In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append(f'[Time: {global_time}. Person: {name_two}. Memory: {action_results[name_two]}]\n')

# Rank Memories

In [ ]:
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [ ]:
memories['Daniel']

In [ ]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {locations[name]}. It is currently {global_time}:00. You observe the following: {memory}. Give a rating from 1 (not important) to 5 (very important) of how much you care about this memory given your own plans. Answer only with the number."
    #print('MEMORY for '+name+': '+memory)
    res = generate(prompt)
    #print('RES: '+res)
    rating = get_rating(res)
    #print('RATING: '+str(rating))
    max_attempts = 3
    current_attempt = 1
    while rating is None and current_attempt<max_attempts:
      res = generate(prompt)
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((memory, rating))

for name in town_people.keys():
    print(f'Memories with ratings for {name}\n{memory_ratings[name]}')

# Compress Memories

In [ ]:
MEMORY_LIMIT = 5
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '. '.join([a[0] for a in relevant_memories])
  #print(f'String to compress: {memory_string_to_compress}')
  prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {locations[name]}. It is currently {global_time}:00. You observe the following: {memory_string_to_compress}. Summarize these memories in a few sentences and from your perspective as {name}."
  res = generate(prompt)
  compressed_memories[name] = f'[Recollection at Time {global_time}:00: {res}]'
  print(f'Compressed memory for {name}\n{compressed_memories[name]}')
  compressed_memories_all[name].append(compressed_memories[name])

In [ ]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {locations[name]}. It is currently {global_time}:00. You have the following memories: {compressed_memories[name]}. A location you could be in is {area}, with description {town_areas[area]}. Give a rating between 1 (highly unlikely) and 5 (highly likely) to how likely you are to be at {area} the next hour."
    #print(prompt)
    res = generate(prompt)
    rating = get_rating(res)
    max_attempts = 3
    current_attempt = 1
    while rating is None and current_attempt<max_attempts:
      res = generate(prompt)
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = f'[Recollection at Time {global_time}:00: I moved to {place_ratings_sorted[0][0]}.]'
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


## Continue for more hours

In [ ]:
for repeats in range(6):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {location} with the following description: {town_areas[location]}. Your memories are: {'\n'.join(compressed_memories_all[name][-5:])}. It is currently {global_time}:00. The following people are in this area: {', '.join([p for p in people if p!=name])}. You can interact with them."
      people_description = []
      for i in people:
        if i != name:
          people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(action_prompts[name])
    # Now clean the action
    prompt = f"""
    Convert the following paragraph to first person past tense:
    "{action_results[name]}"
    """
    action_results[name] = generate(prompt).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  talked_to = []
  for name in town_people.keys():
    if name in talked_to:
      continue
    a = action_results[name]
    prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. The following is your recent action: {a}. Does this action imply you talking with anyone? If so, choose from the following list the person you talked to, answer with name only. {', '.join([p for p in people if p!=name])}"
    x = generate(prompt)
    names = [i for i in re.findall('|'.join(town_people.keys()), x)]
    if names:
      prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. The following is your recent action: {a}. Since you talked with {names[0]}, write a transcript of the conversation. Keep it to 300 words."
      c = generate(prompt)
      talked_to.append(names[0])
      chats[name].append((names[0], f'[Time: {global_time}. Chat between {name} and {names[0]}:\n{c}]\n'))
      chats[names[0]].append((name, f'[Time: {global_time}. Chat between {names[0]} and {name}:\n{c}]\n'))
  for name in chats.keys():
    if not chats[name]:
        continue
    prompt = f"You are acting as {name}. {town_people[name]} You are planning: {plans[name]}. You just had the conversation below with {chats[name][-1][0]}. Summarise what you personally got out of it, in one sentence. {chats[name][-1][1]}"
    s = generate(prompt)
    memories[name].append(f'[Time: {global_time}. Person: {chats[name][-1][0]}. Memory: {s}]\n'
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append(f'[Time: {global_time}. Person: {name_two}. Memory: {action_results[name_two]}]\n'

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = f"You are acting as {name}. Your plans are: {plans[name]}. Your memories are: {'\n'.join(compressed_memories_all[name][-7:])}. You are currently in {locations[name]}. It is currently {global_time}:00. You observe the following: {memory}. Give a rating between 1 (unimportant) and 5 (very important), to how much you care about this memory, given your plans."
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 3
      current_attempt = 1
      while rating is None and current_attempt<max_attempts:
        res = generate(prompt)
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((memory, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name], 
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '. '.join([a[0] for a in relevant_memories])
    prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {locations[name]}. It is currently {global_time}:00. You observe the following: {memory_string_to_compress}. Summarize these memories in a few sentences and from your perspective as {name}."
    res = generate(prompt)
    compressed_memories[name] = f'[Recollection at Time {global_time}:00: {res}]'
    print(f'Compressed memory for {name}: {compressed_memories[name]}')
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = f"You are acting as {name}. Your plans are: {plans[name]}. You are currently in {locations[name]}. It is currently {global_time}:00. You have the following memories: {compressed_memories[name]}. A location you could be in is {area}, with description {town_areas[area]}. Give a rating between 1 (highly unlikely) and 5 (highly likely) to how likely you are to be at {area} the next hour."
      res = generate(prompt)
      rating = get_rating(res)
      max_attempts = 3
      current_attempt = 1
      while rating is None and current_attempt<max_attempts:
        res = generate(prompt)
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name], 
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = f'[Recollection at Time {global_time}:00: I moved to {place_ratings_sorted[0][0]}.]'
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


In [ ]:
compressed_memories_all
#memory_string_to_compress